In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

import pandas as pd 
import math

In [2]:
#  Import and read the charity_data.csv.
df = pd.read_csv("Resources/charity_data.csv")
df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
#drop nonmeaningful variables
df = df.drop("EIN",axis=1).drop("NAME",axis=1)
df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [4]:
# APPLICATION_TYPEs with fewer than 15 instances are lumped into other
binneddf = df
counts = df["APPLICATION_TYPE"].value_counts()
for APPLICATION_TYPE in df["APPLICATION_TYPE"].unique():
    if counts[APPLICATION_TYPE] < 15:
        binneddf["APPLICATION_TYPE"] = binneddf["APPLICATION_TYPE"].replace(APPLICATION_TYPE, "other")

binneddf["APPLICATION_TYPE"].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
T12         27
T2          16
other       11
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# CLASSIFICATIONs with fewer than 100 instances are lumped into other
counts = df["CLASSIFICATION"].value_counts()
for CLASSIFICATION in df["CLASSIFICATION"].unique():
    if counts[CLASSIFICATION] < 100:
        binneddf["CLASSIFICATION"] = binneddf["CLASSIFICATION"].replace(CLASSIFICATION, "other")

binneddf["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [6]:
#look at the dataframe so far
binneddf

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [7]:
binneddf2 = binneddf.copy()
# SPECIAL_CONSIDERATIONS is a binary category, so convert it to a single 0 or 1 instead of making 2 dummy variables out of it (one for Y and one for N)
binneddf2["SPECIAL_CONSIDERATIONS"] = binneddf2["SPECIAL_CONSIDERATIONS"].replace("N", 0).replace("Y", 1)
binneddf2

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,0,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,0,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,0,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,0,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,0,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,0,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,0,5000,1


In [8]:
# turn categorical data into numerical data
dummied2 = pd.get_dummies(binneddf2)
dummied2

,STATUS,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T3,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,0,5000,1,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,0,108590,1,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
2,1,0,5000,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,1,0,6692,1,0,0,0,0,0,1,...,1,0,0,1,0,0,0,0,0,0
4,1,0,142590,1,0,0,0,0,0,1,...,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,0,5000,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
34295,1,0,5000,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
34296,1,0,5000,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
34297,1,0,5000,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [9]:
# split and scale the data
X = dummied2.drop("IS_SUCCESSFUL",axis=1)
y = dummied2["IS_SUCCESSFUL"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_scaler = StandardScaler()
X_scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [16]:
# define the topology of the neural network
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=36, activation="relu", input_dim=52))
nn_model.add(tf.keras.layers.Dense(units=20, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=12, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
nn_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 36)                1908      
                                                                 
 dense_9 (Dense)             (None, 20)                740       
                                                                 
 dense_10 (Dense)            (None, 12)                252       
                                                                 
 dense_11 (Dense)            (None, 1)                 13        
                                                                 
Total params: 2,913
Trainable params: 2,913
Non-trainable params: 0
_________________________________________________________________


In [17]:
# make a function to save checkpoints
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath="checkpoints", save_weights_only=True,verbose=1)

In [18]:
#train the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100,callbacks=[cp_callback])

Epoch 1/100
778/804 [============================>.] - ETA: 0s - loss: 0.5769 - accuracy: 0.7143
Epoch 1: saving model to checkpoints
804/804 [==============================] - 1s 962us/step - loss: 0.5756 - accuracy: 0.7154
Epoch 2/100
804/804 [==============================] - ETA: 0s - loss: 0.5541 - accuracy: 0.7290
Epoch 2: saving model to checkpoints
804/804 [==============================] - 1s 849us/step - loss: 0.5541 - accuracy: 0.7290
Epoch 3/100
775/804 [===========================>..] - ETA: 0s - loss: 0.5505 - accuracy: 0.7287
Epoch 3: saving model to checkpoints
804/804 [==============================] - 1s 941us/step - loss: 0.5504 - accuracy: 0.7287
Epoch 4/100
804/804 [==============================] - ETA: 0s - loss: 0.5483 - accuracy: 0.7311
Epoch 4: saving model to checkpoints
804/804 [==============================] - 1s 1ms/step - loss: 0.5483 - accuracy: 0.7311
Epoch 5/100
750/804 [==========================>...] - ETA: 0s - loss: 0.5462 - accuracy: 0.7322
Epoch

775/804 [===========================>..] - ETA: 0s - loss: 0.5362 - accuracy: 0.7377
Epoch 37: saving model to checkpoints
804/804 [==============================] - 1s 938us/step - loss: 0.5364 - accuracy: 0.7376
Epoch 38/100
793/804 [============================>.] - ETA: 0s - loss: 0.5358 - accuracy: 0.7382
Epoch 38: saving model to checkpoints
804/804 [==============================] - 1s 924us/step - loss: 0.5360 - accuracy: 0.7376
Epoch 39/100
802/804 [============================>.] - ETA: 0s - loss: 0.5363 - accuracy: 0.7380
Epoch 39: saving model to checkpoints
804/804 [==============================] - 1s 968us/step - loss: 0.5362 - accuracy: 0.7380
Epoch 40/100
798/804 [============================>.] - ETA: 0s - loss: 0.5359 - accuracy: 0.7376
Epoch 40: saving model to checkpoints
804/804 [==============================] - 1s 977us/step - loss: 0.5359 - accuracy: 0.7378
Epoch 41/100
788/804 [============================>.] - ETA: 0s - loss: 0.5350 - accuracy: 0.7378
Epoch 4

759/804 [===========================>..] - ETA: 0s - loss: 0.5315 - accuracy: 0.7409
Epoch 73: saving model to checkpoints
804/804 [==============================] - 1s 894us/step - loss: 0.5331 - accuracy: 0.7397
Epoch 74/100
803/804 [============================>.] - ETA: 0s - loss: 0.5329 - accuracy: 0.7394
Epoch 74: saving model to checkpoints
804/804 [==============================] - 1s 972us/step - loss: 0.5329 - accuracy: 0.7394
Epoch 75/100
796/804 [============================>.] - ETA: 0s - loss: 0.5325 - accuracy: 0.7409
Epoch 75: saving model to checkpoints
804/804 [==============================] - 1s 1ms/step - loss: 0.5330 - accuracy: 0.7406
Epoch 76/100
763/804 [===========================>..] - ETA: 0s - loss: 0.5333 - accuracy: 0.7392
Epoch 76: saving model to checkpoints
804/804 [==============================] - 1s 1ms/step - loss: 0.5335 - accuracy: 0.7393
Epoch 77/100
759/804 [===========================>..] - ETA: 0s - loss: 0.5328 - accuracy: 0.7396
Epoch 77: s

In [19]:
#assess the model
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
nn_model.save("AlphabetSoupCharity.h5")

268/268 - 0s - loss: 0.5591 - accuracy: 0.7286 - 295ms/epoch - 1ms/step
Loss: 0.5590829849243164, Accuracy: 0.7286297082901001


In [20]:
#prune the lesser-used categories more aggessively
counts = binneddf2["APPLICATION_TYPE"].value_counts()
for APPLICATION_TYPE in binneddf2["APPLICATION_TYPE"].unique():
    if counts[APPLICATION_TYPE] < 100:
        binneddf2["APPLICATION_TYPE"] = binneddf2["APPLICATION_TYPE"].replace(APPLICATION_TYPE, "other")

binneddf2["APPLICATION_TYPE"].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
other      120
Name: APPLICATION_TYPE, dtype: int64

In [21]:
#prune the lesser-used categories more aggessively
counts = binneddf2["CLASSIFICATION"].value_counts()
for CLASSIFICATION in binneddf2["CLASSIFICATION"].unique():
    if counts[CLASSIFICATION] < 300:
        binneddf2["CLASSIFICATION"] = binneddf2["CLASSIFICATION"].replace(CLASSIFICATION, "other")

binneddf2["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [26]:
# turn categorical data into numerical data
dummied2 = pd.get_dummies(binneddf2)
dummied2

,STATUS,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,0,5000,1,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,0,108590,1,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,0,5000,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
3,1,0,6692,1,0,0,1,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,1,0,142590,1,0,0,1,0,0,0,...,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,0,5000,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
34295,1,0,5000,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
34296,1,0,5000,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
34297,1,0,5000,1,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0


In [27]:
# split and scale the data
X = dummied2.drop("IS_SUCCESSFUL",axis=1)
y = dummied2["IS_SUCCESSFUL"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_scaler = StandardScaler()
X_scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [28]:
# define the topology of the neural network
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=36, activation="relu", input_dim=44))
nn_model.add(tf.keras.layers.Dense(units=20, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=12, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
nn_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 36)                1620      
                                                                 
 dense_17 (Dense)            (None, 20)                740       
                                                                 
 dense_18 (Dense)            (None, 12)                252       
                                                                 
 dense_19 (Dense)            (None, 1)                 13        
                                                                 
Total params: 2,625
Trainable params: 2,625
Non-trainable params: 0
_________________________________________________________________


In [29]:
# make a function to save checkpoints
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath="checkpoints", save_weights_only=True,verbose=1)

In [30]:
#train the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100,callbacks=[cp_callback])

Epoch 1/100
779/804 [============================>.] - ETA: 0s - loss: 0.5857 - accuracy: 0.7101
Epoch 1: saving model to checkpoints
804/804 [==============================] - 1s 877us/step - loss: 0.5844 - accuracy: 0.7110
Epoch 2/100
803/804 [============================>.] - ETA: 0s - loss: 0.5563 - accuracy: 0.7280
Epoch 2: saving model to checkpoints
804/804 [==============================] - 1s 911us/step - loss: 0.5562 - accuracy: 0.7281
Epoch 3/100
749/804 [==========================>...] - ETA: 0s - loss: 0.5505 - accuracy: 0.7307
Epoch 3: saving model to checkpoints
804/804 [==============================] - 1s 910us/step - loss: 0.5519 - accuracy: 0.7295
Epoch 4/100
765/804 [===========================>..] - ETA: 0s - loss: 0.5484 - accuracy: 0.7297
Epoch 4: saving model to checkpoints
804/804 [==============================] - 1s 880us/step - loss: 0.5494 - accuracy: 0.7296
Epoch 5/100
755/804 [===========================>..] - ETA: 0s - loss: 0.5481 - accuracy: 0.7315
Epo

749/804 [==========================>...] - ETA: 0s - loss: 0.5389 - accuracy: 0.7373
Epoch 37: saving model to checkpoints
804/804 [==============================] - 1s 899us/step - loss: 0.5384 - accuracy: 0.7378
Epoch 38/100
756/804 [===========================>..] - ETA: 0s - loss: 0.5376 - accuracy: 0.7377
Epoch 38: saving model to checkpoints
804/804 [==============================] - 1s 900us/step - loss: 0.5385 - accuracy: 0.7370
Epoch 39/100
760/804 [===========================>..] - ETA: 0s - loss: 0.5383 - accuracy: 0.7378
Epoch 39: saving model to checkpoints
804/804 [==============================] - 1s 896us/step - loss: 0.5378 - accuracy: 0.7381
Epoch 40/100
759/804 [===========================>..] - ETA: 0s - loss: 0.5385 - accuracy: 0.7372
Epoch 40: saving model to checkpoints
804/804 [==============================] - 1s 899us/step - loss: 0.5381 - accuracy: 0.7376
Epoch 41/100
789/804 [============================>.] - ETA: 0s - loss: 0.5378 - accuracy: 0.7372
Epoch 4

753/804 [===========================>..] - ETA: 0s - loss: 0.5346 - accuracy: 0.7382
Epoch 73: saving model to checkpoints
804/804 [==============================] - 1s 893us/step - loss: 0.5354 - accuracy: 0.7383
Epoch 74/100
765/804 [===========================>..] - ETA: 0s - loss: 0.5359 - accuracy: 0.7396
Epoch 74: saving model to checkpoints
804/804 [==============================] - 1s 886us/step - loss: 0.5353 - accuracy: 0.7399
Epoch 75/100
759/804 [===========================>..] - ETA: 0s - loss: 0.5358 - accuracy: 0.7397
Epoch 75: saving model to checkpoints
804/804 [==============================] - 1s 885us/step - loss: 0.5359 - accuracy: 0.7398
Epoch 76/100
754/804 [===========================>..] - ETA: 0s - loss: 0.5337 - accuracy: 0.7402
Epoch 76: saving model to checkpoints
804/804 [==============================] - 1s 893us/step - loss: 0.5350 - accuracy: 0.7386
Epoch 77/100
768/804 [===========================>..] - ETA: 0s - loss: 0.5364 - accuracy: 0.7384
Epoch 7

In [31]:
#assess the model
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
nn_model.save("AlphabetSoupCharity.h5")

268/268 - 0s - loss: 0.5569 - accuracy: 0.7282 - 279ms/epoch - 1ms/step
Loss: 0.5568958520889282, Accuracy: 0.7281632423400879


In [32]:
# INCOME_AMT is treated as a categorical variable, but is also quantitative in nature, let's try treating it as purely quanitative
binneddf2["INCOME_AMT"] = binneddf2["INCOME_AMT"].replace("0",0)
binneddf2["INCOME_AMT"] = binneddf2["INCOME_AMT"].replace("1-9999",math.log(9999))
binneddf2["INCOME_AMT"] = binneddf2["INCOME_AMT"].replace("10000-24999",math.log(24999))
binneddf2["INCOME_AMT"] = binneddf2["INCOME_AMT"].replace("25000-99999",math.log(99999))
binneddf2["INCOME_AMT"] = binneddf2["INCOME_AMT"].replace("100000-499999",math.log(499999))
binneddf2["INCOME_AMT"] = binneddf2["INCOME_AMT"].replace("1M-5M",math.log(5000000))
binneddf2["INCOME_AMT"] = binneddf2["INCOME_AMT"].replace("5M-10M",math.log(10000000))
binneddf2["INCOME_AMT"] = binneddf2["INCOME_AMT"].replace("10M-50M",math.log(50000000))
binneddf2["INCOME_AMT"] = binneddf2["INCOME_AMT"].replace("50M+",math.log(100000000))

In [33]:
# ASK_AMT varies over multiple orders of magnitude, so the small values get lumped together.
# taking the natural log should make them stand apart more
for ASK_AMT in binneddf2["ASK_AMT"].unique():
    binneddf2["ASK_AMT"] = binneddf2["ASK_AMT"].replace(ASK_AMT,math.log(ASK_AMT))
binneddf2

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0.000000,0,8.517193,1
1,T3,Independent,C2000,Preservation,Co-operative,1,9.210240,0,11.595335,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0.000000,0,8.517193,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10.126591,0,8.808668,1
4,T3,Independent,C1000,Heathcare,Trust,1,13.122361,0,11.867729,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0.000000,0,8.517193,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0.000000,0,8.517193,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0.000000,0,8.517193,0
34297,T5,Independent,C3000,ProductDev,Association,1,0.000000,0,8.517193,1


In [34]:
# turn categorical data into numerical data
dummied2 = pd.get_dummies(binneddf2)
dummied2

,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,...,CLASSIFICATION_other,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust
0,1,0.000000,0,8.517193,1,1,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
1,1,9.210240,0,11.595335,1,0,0,1,0,0,...,0,0,0,0,1,0,0,1,0,0
2,1,0.000000,0,8.517193,0,0,0,0,0,1,...,0,0,0,0,0,1,1,0,0,0
3,1,10.126591,0,8.808668,1,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,1
4,1,13.122361,0,11.867729,1,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,0.000000,0,8.517193,0,0,0,0,1,0,...,0,0,0,0,0,1,1,0,0,0
34295,1,0.000000,0,8.517193,0,0,0,0,1,0,...,0,0,0,0,0,1,1,0,0,0
34296,1,0.000000,0,8.517193,0,0,0,1,0,0,...,0,0,0,0,1,0,1,0,0,0
34297,1,0.000000,0,8.517193,1,0,0,0,0,1,...,0,0,0,0,0,1,1,0,0,0


In [35]:
# split and scale the data
X = dummied2.drop("IS_SUCCESSFUL",axis=1)
y = dummied2["IS_SUCCESSFUL"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_scaler = StandardScaler()
X_scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [36]:
# define the topology of the neural network
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=36, activation="relu", input_dim=36))
nn_model.add(tf.keras.layers.Dense(units=20, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=12, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
nn_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 36)                1332      
                                                                 
 dense_21 (Dense)            (None, 20)                740       
                                                                 
 dense_22 (Dense)            (None, 12)                252       
                                                                 
 dense_23 (Dense)            (None, 1)                 13        
                                                                 
Total params: 2,337
Trainable params: 2,337
Non-trainable params: 0
_________________________________________________________________


In [37]:
# make a function to save checkpoints
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath="checkpoints", save_weights_only=True,verbose=1)

In [38]:
#train the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100,callbacks=[cp_callback])

Epoch 1/100
750/804 [==========================>...] - ETA: 0s - loss: 0.5710 - accuracy: 0.7184
Epoch 1: saving model to checkpoints
804/804 [==============================] - 1s 839us/step - loss: 0.5706 - accuracy: 0.7177
Epoch 2/100
796/804 [============================>.] - ETA: 0s - loss: 0.5525 - accuracy: 0.7273
Epoch 2: saving model to checkpoints
804/804 [==============================] - 1s 855us/step - loss: 0.5527 - accuracy: 0.7271
Epoch 3/100
794/804 [============================>.] - ETA: 0s - loss: 0.5500 - accuracy: 0.7301
Epoch 3: saving model to checkpoints
804/804 [==============================] - 1s 912us/step - loss: 0.5503 - accuracy: 0.7300
Epoch 4/100
766/804 [===========================>..] - ETA: 0s - loss: 0.5494 - accuracy: 0.7291
Epoch 4: saving model to checkpoints
804/804 [==============================] - 1s 882us/step - loss: 0.5490 - accuracy: 0.7293
Epoch 5/100
747/804 [==========================>...] - ETA: 0s - loss: 0.5477 - accuracy: 0.7304
Epo

785/804 [============================>.] - ETA: 0s - loss: 0.5398 - accuracy: 0.7365
Epoch 37: saving model to checkpoints
804/804 [==============================] - 1s 937us/step - loss: 0.5392 - accuracy: 0.7369
Epoch 38/100
787/804 [============================>.] - ETA: 0s - loss: 0.5390 - accuracy: 0.7381
Epoch 38: saving model to checkpoints
804/804 [==============================] - 1s 934us/step - loss: 0.5389 - accuracy: 0.7382
Epoch 39/100
791/804 [============================>.] - ETA: 0s - loss: 0.5384 - accuracy: 0.7376
Epoch 39: saving model to checkpoints
804/804 [==============================] - 1s 1ms/step - loss: 0.5387 - accuracy: 0.7370
Epoch 40/100
803/804 [============================>.] - ETA: 0s - loss: 0.5384 - accuracy: 0.7375
Epoch 40: saving model to checkpoints
804/804 [==============================] - 1s 970us/step - loss: 0.5385 - accuracy: 0.7375
Epoch 41/100
772/804 [===========================>..] - ETA: 0s - loss: 0.5376 - accuracy: 0.7379
Epoch 41:

763/804 [===========================>..] - ETA: 0s - loss: 0.5356 - accuracy: 0.7384
Epoch 73: saving model to checkpoints
804/804 [==============================] - 1s 957us/step - loss: 0.5352 - accuracy: 0.7388
Epoch 74/100
753/804 [===========================>..] - ETA: 0s - loss: 0.5347 - accuracy: 0.7391
Epoch 74: saving model to checkpoints
804/804 [==============================] - 1s 905us/step - loss: 0.5351 - accuracy: 0.7388
Epoch 75/100
767/804 [===========================>..] - ETA: 0s - loss: 0.5343 - accuracy: 0.7408
Epoch 75: saving model to checkpoints
804/804 [==============================] - 1s 960us/step - loss: 0.5353 - accuracy: 0.7400
Epoch 76/100
803/804 [============================>.] - ETA: 0s - loss: 0.5351 - accuracy: 0.7393
Epoch 76: saving model to checkpoints
804/804 [==============================] - 1s 904us/step - loss: 0.5350 - accuracy: 0.7393
Epoch 77/100
772/804 [===========================>..] - ETA: 0s - loss: 0.5350 - accuracy: 0.7391
Epoch 7

In [39]:
#assess the model
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
nn_model.save("AlphabetSoupCharity.h5")

268/268 - 0s - loss: 0.5533 - accuracy: 0.7313 - 279ms/epoch - 1ms/step
Loss: 0.5532986521720886, Accuracy: 0.7313119769096375


In [40]:
nn_model.save("AlphabetSoupCharity_Optimization.h5")

In [41]:
y_test.mean()

0.5343440233236152

In [42]:
0.7313119769096375-0.7265306115150452

0.004781365394592285

# Report

The purpose of this analysis is to explain the process and results of my deep learning models of the data provided.

* The target for the model is IS_SUCCESSFUL.
* The EIN and NAME variables are just unique identifiers for each charity.  They are neither target nor feature, so are removed
* All the other variables besides those three are considered features.

* I chose 3 hidden layers for my models.  For my first model, I chose layers of 53, 20, and 12 neurons for the three layers with the relu function for the hidden layers and the sigmoid function for the output layer.  I wanted an accurate fit without taking too much time to train.  My results did not appear to be particularly sensitive to the number of neurons or layers, always getting about 74% accuracy for training and about 72.6% accuracy for testing.  
*  For my optimized models, I used hidden layers with 36, 20, and 12 neurons.  These models also did not seem particularly sensitive to changes in the topology of the neural network (i.e. I didn't see any improvement for bigger networks, nor did I see an increase in overfitting.)
* My attemts to achieve 75% accuracy were unsuccessful.  I did not even find a model that got 75% accuracy on the training data.  
* My first attempt to improve the model was to turn the binary SPECIAL_CONSIDERATIONS from Y or N (which would then get converted into separate SPECIAL_CONSIDERATIONS_N and SPECIAL_CONSIDERATIONS_Y columns) into a single 0 or 1 column.  My second attempt was to more aggessively lump together the categories of APPLICATION_TYPE and CLASSIFICATION in hopes of reducing the effects of outliers.  My third attempt combined two changes.  One of the columns (INCOME_AMT) is categorical in the original data, but those categories have a definite ordering on the number line, so I turned the column into quantitative data by mapping 0 -> 0, 50M -> ln(100,000,000), and all other categories to the natural log of their upper bound.  I also reduced the variation between the highest values of ASK_AMT and the lowest (and spread out the lower values) by taking the natural log of that column.

Overall, my models showed a consistent level of quality across many variations of inputs.  With only a small difference in accuracy between the training data and the test data, the models were not particularly overfitted.  On the other hand, I struggled to improve my models over the initial one I produced, only getting about half a percentage point above that model in accuracy.  I think this data probably is probably only 74% or so predicted by the features provided, so it behooves us to have a certain level of humility when trying to model it.  Even 73% from my model is substantially better than always guessing 1, which would yield an accuracy around 53% for the test data, so my model does provide some predictive power.